<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/notebooks/05-CNN-I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Clasificación con Redes Neuronales Convolucionales</h1>

En esta notebook usaremos una red neuronal convolucional (CNN) para un problema de clasificación de imágenes, además, compararemos esta arquitectura con la arquitectura MLP.

<p align="center">
  <img src="https://drive.google.com/uc?id=1bzFBdsAq40yN95k2pA5X2OGsXf-40v0t" width="600" />
</p>

El CIFAR-10 (Canadian Institute For Advanced Research) es un conjunto de datos clásico en visión por computadora, compuesto por 60,000 imágenes en color (32x32 píxeles) distribuidas en 10 clases distintas: avión, automóvil, pájaro, gato, ciervo, perro, rana, caballo, barco y camión. Cada clase contiene 6,000 imágenes, con 5,000 para entrenamiento y 1,000 para prueba. Su tamaño reducido y su diversidad lo convierten en un benchmark ideal para comparar modelos de aprendizaje automático, especialmente en tareas de clasificación de imágenes. A diferencia de MNIST, donde las MLP pueden alcanzar altos rendimientos, CIFAR-10 presenta mayores desafíos debido a variaciones en iluminación, trasfondos y perspectivas, lo que resalta la ventaja de arquitecturas basadas en convoluciones (CNN).

Su objetivo principal es servir como punto de referencia para evaluar la capacidad de los modelos para generalizar características visuales jerárquicas y espaciales.

[Fuente oficial](https://www-cs-toronto-edu.translate.goog/~kriz/cifar.html?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=tc)

\\

| **Clase**  | **Descripción**       |  
|------------|-----------------------|  
| 0          | Avión (airplane)      |  
| 1          | Automóvil (automobile)|  
| 2          | Pájaro (bird)         |  
| 3          | Gato (cat)            |  
| 4          | Ciervo (deer)         |  
| 5          | Perro (dog)           |  
| 6          | Rana (frog)           |  
| 7          | Caballo (horse)       |  
| 8          | Barco (ship)          |  
| 9          | Camión (truck)        |  

Verifiquemos que el entorno de ejecución en Colab sea GPU

In [ ]:
import tensorflow as tf

print('GPU presente en: {}'.format(tf.test.gpu_device_name()))

In [ ]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

🔵 Obervar que ahora, cada instancia en el dataset es un tensor 3-dimensional.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

random_idxs = np.random.choice(range(X_train.shape[0]),size=5,replace=True)

fig, axs = plt.subplots(1,5,figsize=(15,5))
for i, idx in enumerate(random_idxs):
  axs[i].imshow(X_train[idx])
  axs[i].title.set_text(f'Label: {y_train[idx,0]}')
  axs[i].axis('off')
plt.show()

Veamos el balanceo de clases

In [ ]:
import matplotlib.pyplot as plt

classes, countings = np.unique(y_train,return_counts=True)

classes_names = ['Avión', 'Automóvil', 'Pájaro', 'Gato', 'Ciervo', 'Perro', 'Rana', 'Caballo', 'Barco', 'Camión']

plt.figure()
plt.bar(classes, countings)
plt.xlabel('Clase')
plt.ylabel('Cantidad')
plt.title('Distribución de clases')
plt.xticks(classes,labels=classes_names,rotation=45)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  stratify=y_train,
                                                  test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")

Hacemos el preprocesamiento usual

In [ ]:
print(f"Rango de valores de X_train: {X_train.min()} - {X_train.max()}")
print(f"Rango de valores de X_val: {X_val.min()} - {X_val.max()}")
print(f"Rango de valores de X_test: {X_test.min()} - {X_test.max()}")

In [ ]:
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print(f"Rango de valores de X_train: {X_train.min()} - {X_train.max()}")
print(f"Rango de valores de X_val: {X_val.min()} - {X_val.max()}")
print(f"Rango de valores de X_test: {X_test.min()} - {X_test.max()}")

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from seaborn import heatmap

def plot_training_curves(history):
    plt.figure(figsize=(11,5))
    plt.subplot(1,2,1)
    plt.title("Validation and Training Loss",fontsize=14)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.legend()
    plt.subplot(1,2,2)
    plt.title("Validation and Training Accuracy",fontsize=14)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='validation')
    plt.legend()
    plt.show()

def evaluate(model,X,y):
    y_pred_proba = model.predict(X)
    y_pred = np.argmax(y_pred_proba,axis=1)
    print(f"Accuracy: {accuracy_score(y,y_pred)}")
    print(f"F1 Score: {f1_score(y,y_pred,average='macro')}")
    cm = confusion_matrix(y_pred=y_pred,y_true=y)
    plt.figure()
    heatmap(cm,
            fmt='g',
            annot=True,
            xticklabels=classes_names,
            yticklabels=classes_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

## Modelo MLP

Probemos dos modelos

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input, BatchNormalization, Dropout
from keras.optimizers import Adam

def build_model(input_shape,small=True):
    if small:
        model = Sequential()
        model.add(Input(shape=input_shape))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dense(10, activation='softmax'))
        model.compile(optimizer=Adam(learning_rate=1e-4),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    else:
        model = Sequential()
        model.add(Input(shape=input_shape))
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(256, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        model.add(Dense(10, activation='softmax'))
        model.compile(optimizer=Adam(learning_rate=1e-4),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    return model

###  MLP 1

In [ ]:
model_mlp_1 = build_model((32,32,3),small=True)
model_mlp_1.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=1e-6)

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               restore_best_weights=True)

⌚ El entrenamiento tarda alrededor de 2 minutos

In [ ]:
history = model_mlp_1.fit(X_train, y_train,
                        batch_size=64,
                        epochs=100,
                        validation_data=(X_val, y_val),
                        callbacks=[reduce_lr, early_stopping])

In [ ]:
plot_training_curves(history)

🔵 En estas curvas es muy notorio la acción del callback `ReduceLROnPlateau`.

Evaluemos el modelo en el conjunto de prueba

In [ ]:
model_mlp_1.evaluate(X_test,y_test)

Hagamos una evaluación más detallada

In [ ]:
evaluate(model_mlp_1,X_test,y_test)

### MLP 2

Probemos con una arquitectura MLP más compleja (observa el número de capas, tamaño y número total de parámetros)

In [ ]:
model_mlp_2 = build_model((32,32,3),small=False)
model_mlp_2.summary()

🔵 Observa que estamos usando los mismos callbacks inicializados del entrenamiento pasado. ¿Es esto correcto?

⌚ Este entrenamiento tarda alrededor de 4 minutos con GPU

In [ ]:
history = model_mlp_2.fit(X_train, y_train,
                        batch_size=64,
                        epochs=100,
                        validation_data=(X_val, y_val),
                        callbacks=[reduce_lr, early_stopping])

In [ ]:
plot_training_curves(history)

Tenemos un mejor entrenamiento y un desempeño ligeramente superior

In [ ]:
model_mlp_2.evaluate(X_test,y_test)

In [ ]:
evaluate(model_mlp_2,X_test,y_test)

## Modelo CNN

Definamos ahora un modelo con arquitectura CNN. Usaremos las capas [`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/) para las operaciones de convolución y [`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/) para el pooling.

<p align="center">
  <img src="https://drive.google.com/uc?id=1bzFBdsAq40yN95k2pA5X2OGsXf-40v0t" width="600" />
</p>


**Importante**: Observa la elección de los hiperparámetros `padding="same"` y `strides=1`. Esta elección asegura que las salidas de cada capa convolucional tenga las mismas dimensiones que las entradas.

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input
from keras.models import Sequential

model_cnn = Sequential()
model_cnn.add(Input(shape=(32, 32, 3)))
#----- PARTE CONVOLUCIONAL -------
model_cnn.add(Conv2D(32, 3, activation='relu',
                     padding="same",
                     strides=1))
model_cnn.add(MaxPooling2D())
model_cnn.add(Conv2D(64, 3, activation='relu',
                     padding="same", # Probemos comentando esta línea
                     strides=1))
model_cnn.add(MaxPooling2D())
model_cnn.add(Conv2D(128, 3, activation='relu',
                     padding="same",
                     strides=1))
model_cnn.add(MaxPooling2D())
#----- PARTE MLP ----------
model_cnn.add(Flatten())
model_cnn.add(Dense(512, activation='relu'))
model_cnn.add(Dense(10, activation='softmax'))  # recordar que esta capa está fija por el problema

model_cnn.summary()

In [ ]:
from keras.optimizers import Adam

model_cnn.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

⌚ El entrenamiento tarda alrededor de 3 minutos

In [ ]:
history = model_cnn.fit(X_train, y_train,
                        batch_size=64,
                        epochs=100,
                        validation_data=(X_val, y_val),
                        callbacks=[reduce_lr, early_stopping])

In [ ]:
plot_training_curves(history)

In [ ]:
model_cnn.evaluate(X_test,y_test)

In [ ]:
evaluate(model_cnn,X_test,y_test)

🔵 Observar que tenemos un rendimiento muy superior con esta red CNN al modelo MLP 2, aún cuando el MLP es más grande que el CNN (3.8 millones de parámetros vs 1.1 millones de parámetros). Esto exhibe el hecho de que las redes CNN son arquitecturas especialidas en problemas relacionados con imágenes.